## Introduction

In this notebook, we will show how to build a protein patch GNN using the software library we have built.

## Loading proteins as graph objects in memory

The first thing we do is to load a PDB file into memory as a NetworkX graph object. The tool that we use is called `proteingraph`, and is installable directly from PyPI. The next generation version of `proteingraph` is called `graphein`, and is designed to handle the conversion of multiple types of biological data into its graph representation.

In any case, to load a PDB file of a protein structure into a NetworkX graph object, we use the `read_pdb` function from `proteingraph`.

In [ ]:
from proteingraph import read_pdb
from pyprojroot import here

hiv_graph = read_pdb(here() / "data/hiv1_homology_model.pdb")
hiv_graph

`hiv_graph` is a NetworkX graph object, as you can see from above, and as such will work with the NetworkX API seamlessly.

In [ ]:
hiv_graph.nodes()

## Generating protein patches from a NetworkX graph object

Our `patch_gnn` package contains the functions necessary to generate graph "patches" from a graph. Basically, these graph patches are subgraphs of the larger protein graph, and are defined for a radius around every amino acid in the graph.

In [ ]:
from patch_gnn.graph import generate_patches
graph_patches = generate_patches(hiv_graph, 3)

We can visulaize a few of them to see what they look like.

In [ ]:
import networkx as nx

nx.draw(graph_patches[0])

In [ ]:
nx.draw(graph_patches[-10])

## Converting the graph object into array format

Data preparation is key for us to be able to use the neural network layers defined in `patch_gnn`. In particular, we need to be able to describe amino acids according to numerical molecular descriptors. This will give us the "node feature" matrix that exists for each patch.

### EXPASY molecular descriptors

We have downloaded molecular descriptors from EXPASY, which gives us a large table of 61 descriptors for each of the 20 standard amino acids. From this, we will generate a "node feature matrix", in which each row is one amino acid in the protein patch, and each column is a particular molecular descriptor.

In [ ]:
import pandas as pd
aa_feats = pd.read_csv(here() / "data/amino_acid_properties.csv", index_col=0)
aa_feats

### Generating graph feature tensors

We define a function that will be compatible with `patch_gnn`'s machinery to annotate every amino acid and get back a human-friendly, inspectable dataframe version of the node feature matrix.

In [ ]:
def featurize_amino_acid(n, d, aa_feats: pd.DataFrame) -> pd.Series:
    """
    Featurize a single amino acid.
    
    :param n: Graph node.
    :param d: Graph node attributes.
    :param aa_feats: Dataframe containing amino acid features.
    """
    aa = d["residue_name"]
    feats = pd.Series(aa_feats[aa], name=n)
    return feats

Next up, we collect the node featurization functions into a list.
We must use `partial` to enure that each function's signature is limited to `n, d`.

In [ ]:
from functools import partial
feature_funcs = [partial(featurize_amino_acid, aa_feats=aa_feats)]

Now, we stack the feature tensors for all graph patches (they are individual graph objects themselves) together using the `stack_feature_tensors` function.

In [ ]:
from patch_gnn.graph import stack_feature_tensors
Fs = stack_feature_tensors(graph_patches, funcs=feature_funcs)
Fs.shape

### Generating graph adjacency tensors

The graph adjacency tensors form the diffusion matrix that is used for the message passing operation in graph neural networks. We are free to define any kind of diffusion matrix, as long as they are square matrices.

The semantic meaning of each diffusion matrix may vary. For example, the adjacency matrix + identity matrix put together serves to express the idea of adding "myself + my neighbors" together, while the adjacency matrix alone expresses "give me my neighbors' messages".

Knowing what diffusion matrices to use is likely an area of active research; with `patch_gnn`, we simply assume that there are a ton of them that can be used, and that we linearly combine their results together.

In [ ]:
from functools import partial
import numpy as np
from patch_gnn.graph import (
    identity_matrix, 
    adjacency_matrix, 
    laplacian_matrix,
    to_adjacency_xarray,
    stack_adjacency_tensors
)

adjacency_funcs = []
for i in range(3):
    adjacency_funcs.append(partial(adjacency_matrix, power=i, name=f"adjacency_{i}"))
adjacency_funcs.extend(
    [
        identity_matrix,
        laplacian_matrix,
    ]
)

In [ ]:
As = stack_adjacency_tensors(graph_patches, funcs=adjacency_funcs)

## Neural network models

We can now embark on writing neural network models that take both the `As` and `Fs` stacked up together.

### Embedding layer

Firstly, we might want a custom graph embedding layer.

Here, what we do is stack together a message passing layer,
followed by Dense-Simgoid transformation,
followed by a graph summation op,
then another linear projection to 256 dimensions.

In [ ]:
from jax.experimental import stax
from patch_gnn.layers import MessagePassing, GraphSummation

def CustomGraphEmbedding(n_output: int):
    """Return an embedding of a graph in n_output dimensions."""
    init_fun, apply_fun = stax.serial(
        MessagePassing(),
        stax.Dense(2048),
        stax.Sigmoid,
        GraphSummation(),
        stax.Dense(n_output),
    )
    return init_fun, apply_fun

Just to verify that everything works properly:

In [ ]:
embedding_init_fun, embedding_apply_fun = CustomGraphEmbedding(256)

### Top models

Then, we might want to stack it together with a Linear regression model (for unbounded prediction) or logistic regression layer (for binary classification problems). Here's two examples that we can compose together:

In [ ]:
from jax.random import PRNGKey

def LinearRegression(num_outputs):
    """Linear regression layer."""
    init_fun, apply_fun = stax.serial(
        stax.Dense(num_outputs),
    )
    return init_fun, apply_fun

def LogisticRegression(num_outputs):
    """Logistic regression layer."""
    init_fun, apply_fun = stax.serial(
        stax.Dense(num_outputs),
        stax.Softmax,
    )
    return init_fun, apply_fun

### Composing a full model

Let's now compose the full model together.

In [ ]:
model_init_fun, model_apply_fun = stax.serial(
    CustomGraphEmbedding(256),
    LinearRegression(1),
)

And while we're at it, initialize a random set of parameters.

In [ ]:
output_shape, params = model_init_fun(PRNGKey(42), input_shape=(*Fs[0].shape, As[0].shape[-1]))

### Testing the model

We test the model to make sure data can be passed through it and that we get the right shape at the end.

In [ ]:
from jax import vmap
inputs = (As, Fs)
out = vmap(partial(model_apply_fun, params))(inputs)
out.shape

### Testing the embedding

We do the same with the embedding, this time passing in the first parameter element (since our model is composed of two high-level elements that themselves are potentially nested).

In [ ]:
embedding = vmap(partial(embedding_apply_fun, params[0]))(inputs)
embedding.shape

## Test-driving a learning task

We're now going to set up a dummy learning task. In this learning task, we'll see whether we can learn a random number mapping for each graph.

In [ ]:
from jax import random

outputs = random.normal(PRNGKey(42), shape=(len(graph_patches), 1))

Now, let's try to train a model to predict (ahem, memorize) these numbers from the graph.

We start by test-driving the loss, meticulously making sure that it returns a scalar only.

In [ ]:
from patch_gnn.training import mseloss
from jax import grad

mseloss(params, model_apply_fun, inputs, outputs)

Now, we write the training loop.

In [ ]:
from jax.experimental.optimizers import adam
from patch_gnn.training import mseloss
from jax import grad

dmseloss = grad(mseloss)


In [ ]:
def step(i, state, dloss_fun, apply_fun, update_fun, get_params, inputs, outputs):
    params = get_params(state)
    g = dloss_fun(params, apply_fun, inputs, outputs)
    state = update_fun(i, g, state)
    return state

In [ ]:
from jax import jit
from typing import Tuple

init, update, get_params = adam(step_size=1e-3)
get_params = jit(get_params)
state = init(params)

random_training_step = partial(step, dloss_fun=dmseloss, apply_fun=model_apply_fun, update_fun=update, get_params=get_params, inputs=inputs, outputs=outputs)
random_training_step = jit(random_training_step)

### For-loop based training loop

In [ ]:
from tqdm.autonotebook import tqdm
for i in tqdm(range(100)):
    state = random_training_step(i, state, inputs=inputs, outputs=outputs)

In [ ]:
final_params = get_params(state)

In [ ]:
mseloss(final_params, model_apply_fun, inputs, outputs)

### `lax.scan`-based training loop

In [ ]:
from jax import lax


def make_step_scannable(get_params_func, dloss_func, model_func, update_func, inputs, outputs):
    def inner(previous_state, iteration):
        new_state = step(
            i=iteration,
            state=previous_state,
            get_params=get_params_func,
            dloss_fun=dloss_func,
            apply_fun=model_func,
            update_fun=update_func,
            inputs=inputs,
            outputs=outputs,
        )
        return new_state, previous_state

    return inner
from jax.experimental.optimizers import adam

adam_init, adam_update, adam_get_params = adam(0.005)

step_scannable = make_step_scannable(adam_get_params, dmseloss, model_apply_fun, adam_update, inputs, outputs)

initial_state = adam_init(params)

final_params, history = lax.scan(step_scannable, initial_state, np.arange(1000))

In [ ]:
final_params